In [1]:
import pandas as pd
import dask.dataframe as dd
from dask.diagnostics import ProgressBar


## 데이터 검토

In [2]:
# 367	건축물대장	층별개요 (2022년 07월)	671.04	2022-08-24

ddf = dd.read_parquet("floor")
ddf.head()


,관리_건축물대장_PK,대지_위치,도로명_대지_위치,건물_명,시군구_코드,법정동_코드,대지_구분_코드,번,지,특수지_명,...,구조_코드_명,기타_구조,주_용도_코드,주_용도_코드_명,기타_용도,면적(㎡),주_부속_구분_코드,주_부속_구분_코드_명,면적_제외_여부,생성_일자
0,29155-15130,광주광역시 남구 주월동 997-12번지,광주광역시 남구 군분로77번안길 23,None,29155,10600,0,0997,0012,None,...,벽돌구조,세벽,03001,소매점,점포,69.69,0,주건축물,None,20100105
1,29155-15130,광주광역시 남구 주월동 997-12번지,광주광역시 남구 군분로77번안길 23,None,29155,10600,0,0997,0012,None,...,벽돌구조,세벽,01001,단독주택,주택,69.69,0,주건축물,None,20100105
2,30200-2142,대전광역시 유성구 교촌동 638-2번지,대전광역시 유성구 교촌로6번길 45-14,None,30200,10200,0,0638,0002,None,...,철근콘크리트구조,철근콘크리트조,11199,기타아동관련시설,아동시설,177.95,0,주건축물,None,20100105
3,30200-2142,대전광역시 유성구 교촌동 638-2번지,대전광역시 유성구 교촌로6번길 45-14,None,30200,10200,0,0638,0002,None,...,철근콘크리트구조,철근콘크리트조,11199,기타아동관련시설,아동시설,168.05,0,주건축물,None,20100105
4,30200-2142,대전광역시 유성구 교촌동 638-2번지,대전광역시 유성구 교촌로6번길 45-14,None,30200,10200,0,0638,0002,None,...,벽돌구조,조적조,10999,기타교육연구시설,교육연구 및 복지시설,133.37,0,주건축물,None,20100105


In [3]:
ddf.columns

Index(['관리_건축물대장_PK', '대지_위치', '도로명_대지_위치', '건물_명', '시군구_코드', '법정동_코드',
       '대지_구분_코드', '번', '지', '특수지_명', '블록', '로트', '새주소_도로_코드', '새주소_법정동_코드',
       '새주소_지상지하_코드', '새주소_본_번', '새주소_부_번', '동_명', '층_구분_코드', '층_구분_코드_명',
       '층_번호', '층_번호_명', '구조_코드', '구조_코드_명', '기타_구조', '주_용도_코드', '주_용도_코드_명',
       '기타_용도', '면적(㎡)', '주_부속_구분_코드', '주_부속_구분_코드_명', '면적_제외_여부', '생성_일자'],
      dtype='object')

In [4]:
len(ddf)

20563496

전국 건축물 대상 총 2056만 층

In [5]:
ddf.groupby(["층_구분_코드", "층_구분_코드_명"])['관리_건축물대장_PK'].count().compute()


층_구분_코드  층_구분_코드_명
10       지하            1502171
20       지상           18651744
30       옥탑             408789
40       각층                217
22       복수층(상층)             6
21       복수층(하층)             6
Name: 관리_건축물대장_PK, dtype: int64

In [6]:
ddf_ugrnd = ddf[(ddf["층_구분_코드"] == "10")]
len(ddf_ugrnd)

1502171

In [7]:
# 11 서울 41 경기 28 인천
ddf_met = ddf_ugrnd[ddf_ugrnd["시군구_코드"].str.contains("^11|^41|^28")]
ddf_notmet = ddf_ugrnd[~ddf_ugrnd["시군구_코드"].str.contains("^11|^41|^28")]
len(ddf_met), len(ddf_notmet)

(968715, 533456)

In [8]:
with ProgressBar():
    ddf_met.to_csv("underground_met.csv", single_file=True)
    ddf_notmet.to_csv("underground_notmet.csv", single_file=True)

[########################################] | 100% Completed | 48.25 s
[########################################] | 100% Completed | 44.03 s


In [9]:
sorted = ddf_ugrnd.sort_values("시군구_코드", npartitions=10)

In [10]:
with ProgressBar():
    sorted.to_csv("underground_floor/uground-*.csv")

[########################################] | 100% Completed | 65.52 s
